
## import moduels

In [1]:
import torch
import torch.nn as nn
from torch.optim import *
from torch.optim.lr_scheduler import *
import torch.nn.functional as F

import random
import time
from datetime import timedelta
import shutil

import numpy as np

import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from pytorch_model_summary import summary

from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

import multiprocessing
import timm

from PIL import Image

import os
import json

import pandas as pd
import matplotlib.pyplot as plt
from torchvision.utils import make_grid

from sklearn.model_selection import train_test_split

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [2]:
class CustomTestDataset(Dataset):
    def __init__(self, img_paths:list, transforms=None):
        self.img_paths = img_paths
        self.transforms = transforms
        assert self.transforms is not None, 'you must use transforms in Testset'

    def __getitem__(self, index):
        img = Image.open(self.img_paths[index])
        img = np.array(img)
        img = self.transforms(image=img)["image"]
        return img

    def __len__(self):
        return len(self.img_paths)

## mask ensemble

In [3]:
mask_resnet_path = './checkpoints/mask1/best.pth'
mask_resnet = timm.create_model('resnet34', pretrained=True, num_classes=3)
mask_resnet.to(device)
mask_resnet.load_state_dict(torch.load(mask_resnet_path))

<All keys matched successfully>

In [4]:
mask_eff_path = './checkpoints/mask2/best.pth'
mask_eff = timm.create_model('efficientnet_b4', pretrained=True, num_classes=3)
mask_eff.to(device)
mask_eff.load_state_dict(torch.load(mask_eff_path))

<All keys matched successfully>

In [5]:
mask_cvn_path = './checkpoints/mask3/best.pth'
mask_cvn = timm.create_model('convnext_tiny', pretrained=True, num_classes=3)
mask_cvn.to(device)
mask_cvn.load_state_dict(torch.load(mask_cvn_path))

<All keys matched successfully>

In [6]:
mask_transforms = A.Compose([A.CenterCrop(height=450,width=250),
                             A.Resize(224, 224),
                             A.CLAHE(p=1.),
                             A.ToGray(p=1.),
                             A.Normalize(mean=(0.560,0.524,0.501), std=(0.233,0.243,0.246)),
                             ToTensorV2()])

In [7]:
test_csv_path = './dataset/eval/info.csv'
test_dir = './dataset/eval/images/'
info = pd.read_csv(test_csv_path)
test_imgs_path = [os.path.join(test_dir, img_id) for img_id in info.ImageID]

In [8]:
test_dset = CustomTestDataset(test_imgs_path, mask_transforms)

In [9]:
test_loader = DataLoader(
        test_dset,
        batch_size=256*3,
        num_workers=multiprocessing.cpu_count() // 2,
        shuffle=False,
        pin_memory=use_cuda,
        drop_last=False,
    )

In [10]:
mask_preds = []
for model in [mask_resnet, mask_eff, mask_cvn]:
    model_preds = []
    with torch.no_grad():
            for idx, images in enumerate(test_loader):
                images = images.to(device)
                logits = model(images)
                preds = torch.argmax(logits, dim=-1)
                model_preds+=preds.cpu().numpy().tolist()
    mask_preds.append(model_preds)

In [11]:
m = torch.tensor(mask_preds)
voted_mask = m.mode(0).values.tolist()
print(len(voted_mask))

12600


## gender ensemble

In [12]:
gender_resnet_path = './checkpoints/gender_cls1/best.pth'
gender_resnet = timm.create_model('resnet34', pretrained=True, num_classes=2)
gender_resnet.to(device)
gender_resnet.load_state_dict(torch.load(gender_resnet_path))

<All keys matched successfully>

In [13]:
gender_eff_path = './checkpoints/gender_cls2/best.pth'
gender_eff = timm.create_model('efficientnet_b4', pretrained=True, num_classes=2)
gender_eff.to(device)
gender_eff.load_state_dict(torch.load(gender_eff_path))

<All keys matched successfully>

In [14]:
gender_cvn_path = './checkpoints/gender_cls3/best.pth'
gender_cvn = timm.create_model('convnext_tiny', pretrained=True, num_classes=2)
gender_cvn.to(device)
gender_cvn.load_state_dict(torch.load(gender_cvn_path))

<All keys matched successfully>

In [15]:
gender_transforms = A.Compose([
    A.CenterCrop(height=450,width=250),
    A.Resize(224, 224),
    A.CLAHE(p=1.),
    A.Normalize(mean=(0.560,0.524,0.501), std=(0.233,0.243,0.246)),
    ToTensorV2()
    ])

In [16]:
gender_dset = CustomTestDataset(test_imgs_path, gender_transforms)

In [17]:
test_loader = DataLoader(
        gender_dset,
        batch_size=256*3,
        num_workers=multiprocessing.cpu_count() // 2,
        shuffle=False,
        pin_memory=use_cuda,
        drop_last=False,
    )

In [18]:
gender_preds = []
for model in [gender_resnet, gender_eff, gender_cvn]:
    model_preds = []
    with torch.no_grad():
            for idx, images in enumerate(test_loader):
                images = images.to(device)
                logits = model(images)
                preds = torch.argmax(logits, dim=-1)
                model_preds+=preds.cpu().numpy().tolist()
    gender_preds.append(model_preds)

In [19]:
g = torch.tensor(gender_preds)
voted_gender = g.mode(0).values.tolist()
print(len(voted_gender))

12600


## age ensemble

In [33]:
age_resnet_path = './checkpoints/age1/best.pth'
age_resnet = timm.create_model('resnet34', pretrained=True, num_classes=1)
age_resnet.to(device)
age_resnet.load_state_dict(torch.load(age_resnet_path))

<All keys matched successfully>

In [67]:
age_eff_path = './checkpoints/age2/best.pth'
age_eff = timm.create_model('efficientnet_b4', pretrained=True, num_classes=1)
age_eff.to(device)
age_eff.load_state_dict(torch.load(age_eff_path))

<All keys matched successfully>

In [68]:
age_cvn_path = './checkpoints/age3/best.pth'
age_cvn = timm.create_model('convnext_tiny', pretrained=True, num_classes=1)
age_cvn.to(device)
age_cvn.load_state_dict(torch.load(age_cvn_path))

<All keys matched successfully>

In [69]:
age_transforms = A.Compose([A.CenterCrop(height=450,width=250),
                            A.Resize(224, 224),
                            A.CLAHE(p=1.,clip_limit=6.0),
                            A.ToGray(p=1.),
                            A.Normalize(mean=(0.560,0.524,0.501), std=(0.233,0.243,0.246)),
                            ToTensorV2()])

In [70]:
age_dset = CustomTestDataset(test_imgs_path, age_transforms)

In [71]:
test_loader = DataLoader(
        age_dset,
        batch_size=256*2,
        num_workers=multiprocessing.cpu_count() // 2,
        shuffle=False,
        pin_memory=use_cuda,
        drop_last=False,
    )

In [72]:
age_preds = []
for model in [age_cvn]: # age_resnet, age_eff, 
    model_preds = []
    with torch.no_grad():
            for idx, images in enumerate(test_loader):
                images = images.to(device)
                preds = model(images)
                model_preds+=preds.cpu().numpy().tolist()
    age_preds.append(model_preds)

In [73]:
a = torch.tensor(age_preds)

In [74]:
age_preds

[[[57.18925857543945],
  [57.45584487915039],
  [58.43238067626953],
  [54.8019905090332],
  [21.07880401611328],
  [20.34832191467285],
  [55.2609977722168],
  [41.63117980957031],
  [55.26408004760742],
  [19.133255004882812],
  [57.440860748291016],
  [18.995695114135742],
  [19.505029678344727],
  [19.263750076293945],
  [19.53844451904297],
  [57.838775634765625],
  [57.66050720214844],
  [58.28133010864258],
  [58.73499298095703],
  [57.15207290649414],
  [18.329639434814453],
  [58.9305419921875],
  [57.844051361083984],
  [19.003807067871094],
  [58.8768310546875],
  [20.66802406311035],
  [57.31682205200195],
  [58.56703567504883],
  [57.78886795043945],
  [18.87396240234375],
  [21.70694351196289],
  [58.7401123046875],
  [20.170639038085938],
  [42.835601806640625],
  [58.355655670166016],
  [19.260807037353516],
  [20.30609703063965],
  [58.867435455322266],
  [57.89921569824219],
  [18.736644744873047],
  [59.060062408447266],
  [54.87904739379883],
  [57.49591064453125],


In [75]:
avg_a = torch.mean(a, axis=0)
avg_a.size() # torch.Size([12600	1])

torch.Size([12600, 1])

In [76]:
def age_to_class(age):
    cls = None
    if age<30:
        cls = 0
    elif 30<=age<60:
        cls = 1
    elif age>=60:
        cls = 2
    else:
        cls = None
    if cls == None:
        assert ValueError
    return cls

In [77]:
avg_age = []

for a in avg_a:
    avg_age.append(age_to_class(a))

In [78]:
pd.Series(avg_age).value_counts()

1    7193
0    5407
dtype: int64

In [79]:
# avg_age

[1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,


```
1    7231
0    5368
2       1
dtype: int64
```

In [80]:
def get_label(mask_state, gender, age):
    # gender = gender.lower()
    label = 0
    if mask_state == 0 and gender == 0 and age==0: # 0:
        label = 0
    elif mask_state == 0 and gender == 0 and age==1: # 1
        label = 1
    elif mask_state == 0 and gender == 0 and age==2: # 2
        label = 2
    elif mask_state == 0 and gender == 1 and age==0: # 3
        label = 3
    elif mask_state == 0 and gender == 1 and age==1: # 4
        label = 4
    elif mask_state == 0 and gender == 1 and age==2: # 5
        label = 5
    elif mask_state == 1 and gender == 0 and age==0: # 6
        label = 6
    elif mask_state == 1 and gender == 0 and age==1: # 7
        label = 7
    elif mask_state == 1 and gender == 0 and age==2: # 8
        label = 8
    elif mask_state == 1 and gender == 1 and age==0: # 9
        label = 9
    elif mask_state == 1 and gender == 1 and age==1: # 10
        label = 10
    elif mask_state == 1 and gender == 1 and age==2: # 11
        label = 11
    elif mask_state == 2 and gender == 0 and age==0: # 12
        label = 12
    elif mask_state == 2 and gender == 0 and age==1: # 13
        label = 13
    elif mask_state == 2 and gender == 0 and age==2: # 14
        label = 14
    elif mask_state == 2 and gender == 1 and age==0: # 15
        label = 15
    elif mask_state == 2 and gender == 1 and age==1: # 16
        label = 16
    elif mask_state == 2 and gender == 1 and age==2: # 17
        label = 17
    else:
        raise ValueError
    return label

In [81]:
preds = []
for m,g,a in zip(voted_mask, voted_gender,avg_age):
    label = get_label(m,g,a)
    preds.append(label)

In [82]:
df = pd.read_csv('./dataset/eval/info.csv')
df['ans'] = preds

In [83]:
df.to_csv('ensemble2.csv')

## make soup

In [16]:
os.getcwd()

'/opt/ml/repo/level1_imageclassification_cv-level1-cv-06/T4064'

In [17]:
path = []
for i in range(1,6):
    temp = f'./checkpoints/4hd_{i}/best.pth'
    path.append(temp)

path

['./checkpoints/4hd_1/best.pth',
 './checkpoints/4hd_2/best.pth',
 './checkpoints/4hd_3/best.pth',
 './checkpoints/4hd_4/best.pth',
 './checkpoints/4hd_5/best.pth']

In [18]:
import timm
import torch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = timm.create_model(model_name='resnet34', pretrained=False, num_classes=9)
model_dict = model.state_dict()

In [19]:
model_dict = model.state_dict()
soups = {key:[] for key in model_dict}
for i, model_path in enumerate(path):
    weight = torch.load(model_path, map_location = device)
    weight_dict = weight.state_dict() if hasattr(weight, "state_dict") else weight
    for k, v in weight_dict.items():
        soups[k].append(v)
if 0 < len(soups):
    soups = {k:(torch.sum(torch.stack(v), axis = 0) / len(v)).type(v[0].dtype) for k, v in soups.items() if len(v) != 0}
    model_dict.update(soups)
    model.load_state_dict(model_dict)

In [21]:
model
torch.save(model.state_dict(), f'./checkpoints/soup/first_soup.pth')